In [1]:
import pandas
import tensorflow

print(pandas.__version__)
print(tensorflow.__version__)

2.3.0
2.19.0


In [6]:
!pip install matplotlib
import datetime as dt
from pathlib import Path
import os
import time
from datetime import datetime
from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------------- -------------- 5.2/8.1 MB 29.0 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 29.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 31.8 MB/s eta 0:00:00


In [8]:
data_path = Path('\\Users\\USER\\Downloads\\rec_data\\rec_data')
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    # time 형식을 기존과 맞게 고쳐보기
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [9]:
# 이부분을 이 데이터에 맞게 고치기

# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [10]:
data = cleanse_recursive(data, shortest=2, least_click=5)
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [11]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    cutoff_time = final_time - dt.timedelta(days=n_days)

    # Train: cutoff 이전의 데이터
    train = data[data['Time'] < cutoff_time]

    # Test: cutoff 이후의 데이터, 단 아이템은 train에 있던 것만
    test = data[(data['Time'] >= cutoff_time) & (data['ItemId'].isin(train['ItemId']))]

    return train, test

In [12]:
tr, test = split_by_date(data, n_days=1)  # 마지막 이틀만 테스트용으로 분리
tr, val = split_by_date(tr, n_days=1)  # 마지막 이틀만 테스트용으로 분리


In [13]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [14]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 999550
	 Sessions: 6040
	 Items: 3416
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2003-02-26 16:18:03

* valid Set Stats Info
	 Events: 18
	 Sessions: 7
	 Items: 18
	 First Time : 2003-02-27 04:30:55
	 Last Time : 2003-02-27 17:45:48

* test Set Stats Info
	 Events: 43
	 Sessions: 7
	 Items: 43
	 First Time : 2003-02-27 17:50:41
	 Last Time : 2003-02-28 17:49:50



In [15]:
tr.shape

(999550, 4)

In [16]:
tr = cleanse_recursive(tr, shortest=2, least_click=5)
val = cleanse_recursive(val, shortest=2, least_click=1)
test = cleanse_recursive(test, shortest=2, least_click=1)

In [17]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [18]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [19]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
36,1,1836,5,2000-12-31 22:02:52,5
3,1,3408,4,2000-12-31 22:04:35,6
7,1,2804,5,2000-12-31 22:11:59,7
47,1,1207,4,2000-12-31 22:11:59,8
0,1,1193,5,2000-12-31 22:12:40,9


In [20]:
start = tr_dataset.click_offsets[tr_dataset.session_idx[[[0,1,2,3]]]]       # data 상에서 session이 시작된 위치를 가져옵니다.
end = tr_dataset.click_offsets[tr_dataset.session_idx[[0,1,2,3]] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.

start
end

array([ 53, 182, 233, 254], dtype=int32)

In [21]:
(end - start).min() -1

np.int32(20)

In [22]:
inp = tr_dataset.df['item_idx'].values[start + 20]
inp

array([[ 20,  71, 185,  88]])

In [23]:
target = tr_dataset.df['item_idx'].values[start + 20 + 1]
target

array([[ 21,  72, 186,  56]])

In [24]:
class FullSequenceDataLoader:
    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size
        self.sessions = self._build_sessions()

    def _build_sessions(self):
        sessions = []
        for start, end in zip(self.dataset.click_offsets[:-1], self.dataset.click_offsets[1:]):
            session = self.dataset.df['item_idx'].values[start:end]
            if len(session) >= 2:  # 최소 길이 보장
                sessions.append(session)
        return sessions

    def __iter__(self):
        for i in range(0, len(self.sessions), self.batch_size):
            batch = self.sessions[i:i+self.batch_size]
            input_batch = []
            target_batch = []
            for session in batch:
                input_batch.append(session[:-1])   # [A, B, C]
                target_batch.append(session[-1])   # D
            yield input_batch, target_batch

In [25]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [27]:
def create_model(args):
    # input: (batch, sequence_length, num_items)
    inputs = Input(shape=(None, args.num_items))  # 시퀀스 길이는 가변, 배치 크기도 가변
    x = GRU(args.hsz, return_sequences=False)(inputs)  # 마지막 hidden state만 출력
    x = Dropout(args.drop_rate)(x)
    predictions = Dense(args.num_items, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [28]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.1, lr=0.001, epochs=3, k=20)

In [29]:
model = create_model(args)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 3416)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 50)             │       520,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3416)           │       174,216 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 694,416 (2.65 MB)

 Trainable params: 694,416 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = FullSequenceDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        tr_loader = tqdm(train_loader, desc=f'Train Epoch {epoch}')

        for input_seqs, target_items in tr_loader:
            # One-hot encoding
            input_seqs_padded = pad_sequences(input_seqs, padding='pre')  # 또는 'post'
            input_ohe = to_categorical(input_seqs_padded, num_classes=args.num_items)  # (B, T, num_items)
            target_ohe = to_categorical(target_items, num_classes=args.num_items)  # (B, num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy=result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)
        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:.4f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:.4f}")

def get_metrics(data, model, args, k: int):
    dataset = SessionDataset(data)
    loader = FullSequenceDataLoader(dataset, batch_size=args.batch_size)

    recall_list, mrr_list = [], []

    for input_seqs, label in tqdm(loader, desc='Evaluation'):
        input_ohe = to_categorical(input_seqs, num_classes=args.num_items)  # (B, T, num_items)

        pred = model.predict(input_ohe, batch_size=args.batch_size)  # (B, num_items)
        pred_arg = tf.argsort(pred, direction='DESCENDING')

        for i in range(len(label)):
            recall_list.append(recall_k(pred_arg[i], label[i], k))
            mrr_list.append(mrr_k(pred_arg[i], label[i], k))

    return np.mean(recall_list), np.mean(mrr_list)

In [31]:
train_model(model, args)

Train Epoch 1: 95it [05:54,  3.73s/it, accuracy=0.0071192053, train_loss=7.8924074]
Evaluation: 0it [00:00, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.